In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def etf_tool(
    aktuelles_alter,
    renteneintrittsalter,
    lebenserwartung,
    netto_monatlich,
    inflation_prozent,
    rendite_prozent,
    dynamik_anzeigen,
    investment_monatlich,
    sparratensteigerung_prozent
):
    # --- Parameter & Zeiträume ---
    inflationsrate = inflation_prozent / 100
    sparsteigerung = sparratensteigerung_prozent / 100
    r = rendite_prozent / 100 / 12

    sparjahre = renteneintrittsalter - aktuelles_alter
    entnahmejahre = lebenserwartung - renteneintrittsalter
    jahresreihe = np.arange(aktuelles_alter, lebenserwartung + (1/12), 1/12)

    # --- Kaufkraft-Entwicklung (für Entnahme & Plot) ---
    kaufkraftfaktor_pro_jahr = [(1 + inflationsrate) ** jahr for jahr in range(lebenserwartung - aktuelles_alter + 1)]
    kaufkraft_monatlich = [netto_monatlich * f for f in kaufkraftfaktor_pro_jahr]
    netto_rente = kaufkraft_monatlich[sparjahre]
    netto_ende = kaufkraft_monatlich[-1]

    # --- Kapitalaufbau ---
    kapital = []
    investiert = []
    kontostand = 0

    for monat in range(sparjahre * 12):
        jahr = monat // 12
        rate = investment_monatlich * ((1 + sparsteigerung) ** jahr)
        kontostand = kontostand * (1 + r) + rate
        kapital.append(kontostand)
        investiert.append(investiert[-1] + rate if investiert else rate)

    kapital_sparphase = kapital.copy()
    investiert_gesamt = investiert[-1]

    # --- Entnahmephase (immer inflationsangepasst) ---
    steuer_satz = 0.18
    steuerzahlungen = []

    for monat in range(entnahmejahre * 12):
        jahr = monat // 12
        entnahme = netto_monatlich * ((1 + inflationsrate) ** (sparjahre + jahr))

        if kontostand <= 0:
            break

        anteil_gewinn = max(0, 1 - (investiert_gesamt / kontostand))
        steuer = entnahme * anteil_gewinn * steuer_satz
        kontostand = kontostand * (1 + r) - entnahme - steuer
        kapital.append(kontostand)
        steuerzahlungen.append(steuer)

    vererbung = max(0, kapital[-1])

    # --- Kapital-Liste auffüllen (wenn vorher leerläuft) ---
    if len(kapital) < len(jahresreihe):
        kapital += [0] * (len(jahresreihe) - len(kapital))

    # --- Plots ---
    fig, axs = plt.subplots(2, 1, figsize=(11, 9))

    # Kaufkraftplot
    altersreihe = np.arange(aktuelles_alter, lebenserwartung + 1)
    axs[0].plot(altersreihe, kaufkraft_monatlich, label="Benötigtes Netto pro Monat", linewidth=2)
    axs[0].axvline(renteneintrittsalter, linestyle="--", color="red", label=f"Rentenbeginn ({renteneintrittsalter})")
    axs[0].axvline(lebenserwartung, linestyle=":", color="gray", label=f"Lebenserwartung ({lebenserwartung})")
    axs[0].set_title(
        f"Kaufkraftentwicklung ({inflation_prozent:.1f}% p.a.)\n"
        f"Rente: {netto_rente:,.2f} €/Monat | Ende: {netto_ende:,.2f} €/Monat"
    )
    axs[0].set_xlabel("Alter")
    axs[0].set_ylabel("Benötigtes Netto (€)")
    axs[0].grid(True)
    axs[0].legend()

    # Kapitalplot
    axs[1].plot(jahresreihe, kapital, label="Gesamtkapital", linewidth=2)
    axs[1].plot(jahresreihe[:len(kapital_sparphase)], investiert, label="Investiert", linestyle="--")
    axs[1].axvline(renteneintrittsalter, linestyle="--", color="red", label="Rentenbeginn")
    axs[1].axvline(lebenserwartung, linestyle=":", color="gray", label="Lebenserwartung")
    axs[1].set_title(
        f"ETF-Simulation {'(inflationsbereinigt)' if dynamik_anzeigen else '(nominal)'}\n"
        f"Investiert: {investiert_gesamt:,.2f} € | Steuern: {sum(steuerzahlungen):,.2f} € | "
        f"Vererbung: {vererbung:,.2f} €"
    )
    axs[1].set_xlabel("Alter")
    axs[1].set_ylabel("Kapital in €")
    axs[1].grid(True)
    axs[1].legend()

    plt.tight_layout()
    plt.show()


# --- Interaktive Eingaben ---
interact(
    etf_tool,
    aktuelles_alter=widgets.IntSlider(value=28, min=18, max=70, description="Alter"),
    renteneintrittsalter=widgets.IntSlider(value=67, min=40, max=75, description="Rente mit"),
    lebenserwartung=widgets.IntSlider(value=89, min=70, max=100, description="Lebenserwartung"),
    netto_monatlich=widgets.IntSlider(value=2500, min=500, max=10000, step=100, description="Netto (€)"),
    inflation_prozent=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Inflation (%)"),
    rendite_prozent=widgets.FloatSlider(value=6.5, min=0.0, max=15.0, step=0.1, description="Rendite p.a. (%)"),
    dynamik_anzeigen=widgets.Checkbox(value=True, description="Inflation im Plot berücksichtigen"),
    investment_monatlich=widgets.IntSlider(value=300, min=0, max=3000, step=50, description="Investment (€)"),
    sparratensteigerung_prozent=widgets.FloatSlider(value=2.0, min=0.0, max=10.0, step=0.1, description="Dynamik (%)")
);

interactive(children=(IntSlider(value=28, description='Alter', max=70, min=18), IntSlider(value=67, descriptio…